<a href="https://colab.research.google.com/github/AlexeyTri/PROJECT_SCOOL/blob/master/ActualGraphicBASE_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Условия:**
1. наличие AUTODESK DESKTOP
2. доступ ко всем таблицам в режиме РЕДАКТОР
3. все тактические графики должны быть приведены в гугл форму

# Библиотеки

In [4]:
!pip install pygsheets

In [5]:
import pygsheets

In [6]:
import openpyxl
from openpyxl.cell.cell import WriteOnlyCell
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import pandas as pd
import gspread_dataframe as gd

In [7]:
idx = pd.IndexSlice

In [8]:
from google.colab import auth
auth.authenticate_user()
import re
import gspread
from google.auth import default
creds, _ = default()
import numpy as np
gc = gspread.authorize(creds)
from datetime import datetime

# отчетные даты

In [9]:
# старт-финиш проекта
start_project = '01.02.2022'
start_project = pd.to_datetime(start_project)
finish_project = '02.09.2024'
finish_project = pd.to_datetime(finish_project)

In [10]:
# страт-финиш отчетной 
start_week = '30.12.2022'
start_week = pd.to_datetime(start_week)
finish_week = '16.01.2023'
finish_week = pd.to_datetime(finish_week)

In [11]:
# старт-финиш next недели, за отчетной
start_next_week = '17.01.2023'
start_next_week = pd.to_datetime(start_next_week)
finish_next_week = '20.01.2023'
finish__next_week = pd.to_datetime(finish_next_week)

# Предобработка GS: график проектирования, график СМР, график заказчика

## Реестр выдачи РД

https://docs.google.com/spreadsheets/d/1ZuZv1Rx-mp3AS5LFPjbEa-P92Xux_o_rvaOQb6NWXkE/edit#gid=479522789

In [ ]:
# форма для обновления графика заказчика
worksheet_PD = gc.open('Реестр выдачи РД')
graph_PD = worksheet_PD.worksheet('График ПИР ')
graph_PD = graph_PD.get_all_values()
graph_PD = np.array(graph_PD)
graph_PD = pd.DataFrame(graph_PD[7:])
graph_PD

In [ ]:
# форма для обновления графика подрядчика
graph_PD_RD = pd.DataFrame.from_records(worksheet_PD.worksheet('График ПИР ').get_all_values())
graph_PD_RD

## План работ Договор Адамант

https://docs.google.com/spreadsheets/d/1ngWR2k38yG5qDWfwA9LKi4-R9q9PrIHludYb5vSh4Gc/edit#gid=1411311560

In [ ]:
# форма для обновления графика заказчика
worksheet_SMR = gc.open('План работ Договор Адамант')
graf = worksheet_SMR.worksheet("ГРАФИК РАБОТ")
graf = graf.get_all_values()
graf = np.array(graf)
graf =  pd.DataFrame(graf[1:, :])
graf

In [73]:
# форма для заполнения факта
graf_black = graf.copy()

In [ ]:
# форма для обновления графика недели
week_graf = graf.copy()
week_graf = week_graf.loc[idx[2:],idx[:6]]
week_graf.drop([4,5], axis=1, inplace=True)
week_graf.rename(columns = {0:'passwordCOST', 
                       1:'NAME',2:'START', 3:'FINISH', 6:'valueModel'}, 
            inplace = True)
week_graf = week_graf[week_graf['passwordCOST'] > '']
week_graf = week_graf[week_graf['START'] > '']
week_graf = week_graf[week_graf['FINISH'] > '']
week_graf = week_graf.set_index('passwordCOST')
week_graf['valueModel'] = week_graf['valueModel'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))
week_graf = week_graf.fillna(0)
week_graf

In [ ]:
# общий график работ
graf_name = graf[[0,1]][2:]
graf_name.rename(columns={0: 'passwordCOST', 1: 'NAME'}, inplace=True) 
graf_name = graf_name[graf_name['passwordCOST'] != 0]
graf_name = pd.DataFrame(graf_name['NAME'].values, index=graf_name['passwordCOST'], columns=['NAME'])
graf_total = graf.drop(graf.index[1:10], axis=1)
newColumnsIndex = [pd.to_datetime(pd.Series(graf_total.iloc[0].values[1:]).apply(lambda x: str(x)), format='%d.%m.%Y'), graf_total.iloc[1][1:].values]
graf_total.rename(columns={0: 'passwordCOST'}, inplace=True)
graf_total = graf_total.set_index("passwordCOST")
graf_total = pd.DataFrame(graf_total.values, index=graf_total.index, columns=newColumnsIndex)
graf_total.drop(graf_total.index[:2], axis=0, inplace=True)
for i in graf_total.columns:
    graf_total[i] = graf_total[i].map(lambda x: pd.to_numeric(re.sub("[^0-9 +,]", "", x).replace(',','.')))
graf_total.fillna(0, inplace=True)
graf_total.sort_index(axis=0, level=1, ascending=True)
graf_total = graf_total[graf_total.index > '']
graf_total

In [ ]:
# перечень работ -> вгрузка из Plan Radar
plan_PR = worksheet_SMR.worksheet('PR (копия)')
plan_PR = plan_PR.get_all_values()
plan_PR = np.array(plan_PR)
plan_PR = pd.DataFrame(plan_PR)
plan_PR = plan_PR.iloc[1:, [0,7,9]]
plan_PR = pd.DataFrame(plan_PR)
plan_PR

## График по форме заказчика

In [ ]:
# файл загружается на ГУГЛ диск
from google.colab import files
upload = files.upload()

In [ ]:
df_graph = pd.read_excel(open('Сводный план-график 2022 12 28 иса.xlsx', 'rb'))
df_graph

## Реестр замечаний из ACC

In [ ]:
# файл с реестром замечаний на google drive
from google.colab import files
upload = files.upload()

In [ ]:
value_essuse = pd.read_excel(open('Сводка по проблемам-202301200612.xlsx', 'rb'), sheet_name=1)
value_essuse = value_essuse[(value_essuse['Статус'] == 'Открыто') & (value_essuse['Тип'].isin(['Замечание к РД','Замечания к ПД']))]
value_essuse.loc[:, 'Размещение'] =  value_essuse['Размещение'].map(lambda x: str(x).split('.pdf')[0].split('_')[0])
value_essuse = value_essuse.loc[:, idx['Размещение','Тип замечания']]
agg_func_count = {'Тип замечания': 'count'}
value_essuse = value_essuse.groupby(['Размещение','Тип замечания']).agg(agg_func_count)
value_essuse

# **Обновление графика проектирования подрядчика: 'График ПИР '**

* выгрузка реестра замечаний 
* агрегирование данных по разделам
* заполнение графика проектирования

In [67]:
name_RD_issues = pd.Series(value_essuse.index.get_level_values(0)).unique()

In [68]:
graph_PD_RD = pd.DataFrame.from_records(worksheet_PD.worksheet('График ПИР ').get_all_values())

In [69]:
list_PD_RD = graph_PD_RD.iloc[:,2].to_list()

In [ ]:
for i in name_RD_issues:
    try:
        number_row = list_PD_RD.index(i) + 1
        try:
            value_essuse.loc[i, 'Критическое замечание']
            worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS C')
        except: 
            try:
                value_essuse.loc[i, 'Техническая ошибка'].values[0] != 0  
                value_essuse.loc[i, 'Недостаточно данных'].values[0] != 0
                worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS B')
            except:
                try:
                    worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS A') 
                except:
                    pass
    except:
        pass 

# **Обновление графика СМР подрядчика: 'PLAN_WEEK', 'START', 'ГРАФИК РАБОТ'**

* в модели отмечается выполненный элемент
* данные попадают в GS + обновляется формы PLAN RADAR

## срез на отчетную неделю, план на следующую неделю за отчетной, кумулятивная сумма

In [ ]:
graf_week_plan = graf.loc[:, idx[start_week:finish_week,'план']].sum(axis=1).values
graf_week_plan = pd.DataFrame(graf_week_plan,index=graf.index, columns=['PLAN_WEEK'])
graf_week_fakt = graf.loc[:, idx[start_week:finish_week,'факт']].sum(axis=1).values
graf_week_fakt = pd.DataFrame(graf_week_fakt,index=graf.index, columns=['FAKT_WEEK'])

In [ ]:
graf_next_week_plan = graf.loc[:, idx[start_next_week:finish_next_week,'план']].sum(axis=1).values
graf_next_week_plan = pd.DataFrame(graf_next_week_plan,index=graf.index, columns=['PLAN_NEXT_WEEK'])
graf_next_week_fakt = graf.loc[:, idx[start_next_week:finish_next_week,'факт']].sum(axis=1).values
graf_next_week_fakt = pd.DataFrame(graf_next_week_fakt,index=graf.index, columns=['FAKT_NEXT_WEEK'])

In [ ]:
total_plan = graf.loc[:, idx[start_project:finish_week,'план']].sum(axis=1).values
total_plan = pd.DataFrame(total_plan,index=graf.index, columns=['TOTAL_PLAN'])
total_fakt = graf.loc[:, idx[start_project:finish_week,'факт']].sum(axis=1).values
total_fakt = pd.DataFrame(total_fakt,index=graf.index, columns=['TOTAL_FAKT'])

In [ ]:
new_graf_week = week_graf.join([total_plan,total_fakt,graf_week_plan,graf_week_fakt,graf_next_week_plan,graf_next_week_fakt])

In [ ]:
new_graf_week = new_graf_week[['NAME', 'TOTAL_PLAN','TOTAL_FAKT','PLAN_WEEK','FAKT_WEEK','PLAN_NEXT_WEEK','FAKT_NEXT_WEEK']]

In [ ]:
new_graf_week = new_graf_week.loc[(new_graf_week['PLAN_WEEK']>0) | (new_graf_week['PLAN_NEXT_WEEK']>0)]

In [ ]:
new_graf_week['%TOTAl_WEEK'] =( new_graf_week['FAKT_WEEK'] / new_graf_week['PLAN_WEEK']).mul(100).round(2).astype(str).add(' %')

In [ ]:
new_graf_week['%TOTAL'] =( new_graf_week['TOTAL_FAKT'] / new_graf_week['TOTAL_PLAN']).mul(100).round(2).astype(str).add(' %')

In [ ]:
new_graf_week.rename(columns={'NAME':f'WORK on {pd.to_datetime(start_week).date()}  to {pd.to_datetime(finish_week).date()}'}, inplace=True)

In [ ]:
worksheet_SMR.worksheet('PLAN_WEEK').clear()
worksheet_SMR.worksheet('PLAN_WEEK').append_rows([new_graf_week.columns.values.tolist()]+new_graf_week.values.tolist())

In [ ]:
# справочно
dir(worksheet_SMR.worksheet('PLAN_WEEK'))

## данные по стартовым работам -> триггер для Plan Radar

In [ ]:
start = start_project
finish = pd.to_datetime(pd.Timestamp.today().date())
start, finish

In [44]:
finish_yesterday = finish - pd.Timedelta(1, "d")

In [45]:
total_plan = graf_total.loc[:, idx[start:finish,'план']].sum(axis=1).values
cum_sum_yesterday = graf_total.loc[:, idx[start:finish_yesterday,'план']].sum(axis=1).values
sum_today = graf_total.loc[:, idx[finish:finish,'план']].sum(axis=1).values
# total_plan_value = graf.loc[:, idx[start:finish,'план']].sum(axis=1).values - graf.loc[:, idx[start:finish,'план']].sum(axis=1).values

In [48]:
total_plan = pd.DataFrame(total_plan,index=graf_total.index, columns=['cum_sum_today'])
cum_sum_yesterday = pd.DataFrame(cum_sum_yesterday,index=graf_total.index, columns=['cum_sum_yesterday'])
sum_today = pd.DataFrame(sum_today,index=graf_total.index, columns=[f'sum_today {pd.to_datetime(finish).date()}'])
total_plan = total_plan.join([cum_sum_yesterday, sum_today])

In [49]:
total_plan = total_plan[(total_plan[f'sum_today {pd.to_datetime(finish).date()}']>0) & (total_plan['cum_sum_yesterday'] == 0)]

In [52]:
total_plan = total_plan[[f'sum_today {pd.to_datetime(finish).date()}']]

In [54]:
worksheet_SMR.worksheet('START').clear()
gd.set_with_dataframe(worksheet_SMR.worksheet('START'), total_plan,row=1, col=1, include_index=True, include_column_header=True)

## обновление листа ПЛАН-РАБОТ-> внесение факта

In [74]:
list_date = list(graf_black.iloc[0, :].values)
list_name = list(graf_black.iloc[:, 0].values)

In [ ]:
for i in plan_PR.iloc[:, 0]:
    index_in_plan_PR = list(plan_PR.iloc[:, 0]).index(i)
    date_in_plan_PR = plan_PR.iloc[index_in_plan_PR, 2]
    date_in_plan_PR = '.'.join(reversed(str(date_in_plan_PR).split('-')))
    value_in_plan_PR = plan_PR.iloc[index_in_plan_PR, 1]
    value_in_plan_PR = ','.join(value_in_plan_PR.split('.'))

    index_row_in_graf_black = list_name.index(i) + 2
    index_column_in_graf_black = list_date.index(date_in_plan_PR) + 1

    worksheet_SMR.worksheet("ГРАФИК РАБОТ").update_cell(index_row_in_graf_black, index_column_in_graf_black, value_in_plan_PR)

    print(value_in_plan_PR, date_in_plan_PR, i, index_row_in_graf_black, index_column_in_graf_black)

## обновление листа финансирование:

1. внесение плана КС, кумулятивно за отчетный период
2. акутулизация плана платежей
3. внесение сумм по фактически выполненным работам

## графики:

1. СМР план/факт
2. финансирование план/факт
3. освоение план/факт


In [83]:
from pygsheets.chart import ChartType

In [85]:
gz = pygsheets.authorize(service_file='/content/credentials.json')

In [86]:
 gz = gz.open('План работ Договор Адамант')

In [87]:
gz.worksheet_by_title('PLAN_WEEK').add_chart(('A2', 'A44'), [('B2', 'B44')], chart_type=ChartType.LINE, title='MyChart', anchor_cell='A50')

<Chart LINE 'MyChart'>

# **Обновление графика Закачика**

* позиции имеющие шифр обновляются по шифру
* позиции без шифра обновляются после фильтрации по шифру

## предобработка графика Заказчика

In [75]:
#  список из графика заказчика
list2 = df_graph['Unnamed: 13']
list2 = list2.map(lambda z: str(z).lstrip())
list2 = list2.to_list()
list3 = df_graph['Unnamed: 2']
list3 = list3.map(lambda z: str(z).lstrip())
list3 = list3.to_list()

## обновление СМР

* пайплайн по каждому контракту СМР одинаковый

### пересечение списков видов работ

In [ ]:
#  список из шиифров из 'График работ'
list1 = graf.iloc[:, 0].values
list1

In [ ]:
result = list(set(list1) & set(list2))
result

In [ ]:
for i in result:
    df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = (pd.to_numeric(graf.loc[graf.iloc[:,0].map(lambda z: str(z).lstrip()).values == i, 9].values[0].replace(',', '.').replace('%', ''))/100)
    # проверка корректности внесения обновлений
    print({df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: pd.to_numeric(graf.loc[graf.iloc[:,0].map(lambda z: str(z).lstrip()).values == i, 9].values[0].replace(',', '.').replace('%', ''))/100})    

## обновление проектирования

* первый этап - обновить по шифрам
* второй этап - обновить по наименованию работ

In [78]:
perevod = {
    'STATUS A': 1,
    'STATUS B': 0.85,
    'STATUS C': 0.6,
    'WORK': 0.3,
    'START': 0.05,
    'HOLD': 0,
    '': ''
}

In [ ]:
list_PD_ID_1 = graph_PD[graph_PD.iloc[:, 1].values != ''].iloc[:,2]
list_PD_name = graph_PD[graph_PD.iloc[:, 1].values == ''].iloc[:,3]
list_PD = list_PD_ID_1.append(list_PD_name)
list_PD

In [80]:
result_PD_ID = list(set(list_PD) & set(list2))
result_PD_name = list(set(list_PD) & set(list3))

In [ ]:
for i in result_PD_ID:
    df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = perevod[graph_PD.loc[graph_PD.iloc[:,2] == i,7].values[0]]
    # проверка корректности внесения обновлений
    print({df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: perevod[graph_PD.loc[graph_PD.iloc[:,2] == i,7].values[0]]})

In [ ]:
for i in result_PD_name:
    df_graph.loc[df_graph['Unnamed: 2'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = perevod[graph_PD.loc[graph_PD.iloc[:,3] == i,7].values[0]]
    print({df_graph.loc[df_graph['Unnamed: 2'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: perevod[graph_PD.loc[graph_PD.iloc[:,3] == i,7].values[0]]}) 

создание файла с обновленными данными

In [ ]:
df_graph.to_excel('Сводный план-график {} иса.xlsx'.format('11.01.2023'), sheet_name='Обновление от {}'.format('11.01.2023'),index=False)